In [3]:
import cv2
import numpy as np
import argparse
import io
import os
from google.cloud import vision
from google.cloud.vision import types
# Imports the Google Cloud client library
class ImageAnnotator():

    def __init__(self, path_cred=''):

        # Set credentials
        self.path_cred = path_cred
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path_cred
        # Instantiates a client
        self.client = vision.ImageAnnotatorClient()
    
    def get_faces(self, path_img, print_data):

        """ 
        Annonates image at location path_img returns objects in detected in the image
        args:
            path_img = string
            print_data = bool - prints object data if true
        """ 

        # The name of the image file to annotate
    #         file_name = os.path.join(
    #             os.path.dirname(__file__),
    #             path_img)
        file_name =  path_img

        # Loads the image into memory
        with io.open(file_name, 'rb') as image_file:
            content = image_file.read()

            image = types.Image(content=content)

            response = self.client.face_detection(image=image)

            faces = response.face_annotations
            print( 'Number of faces: ', len(faces))
        if print_data:
            print('Number of faces found: {}'.format(len(faces)))
        return faces

    def annotate_image(self, path_img, path_img_an, img_faces):
        # read image
        emo = ['Angry', 'Surprised','Sad', 'Happy', "Under Exposed", "Blurred", "Headwear"]
        likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                            'LIKELY', 'VERY_LIKELY')
        string = 'No sentiment'
        img = cv2.imread(path_img)
        if len(img_faces)>0:
            for face in img_faces:
                # get vertices
    #                 vertices = []
                x = face.bounding_poly.vertices[0].x
                y = face.bounding_poly.vertices[0].y
                x2 = face.bounding_poly.vertices[2].x
                y2 = face.bounding_poly.vertices[2].y
                img_an = cv2.rectangle(img, (x, y), (x2, y2), (0, 255, 0), 2)

                sentiment = [likelihood_name[face.anger_likelihood],
                            likelihood_name[face.surprise_likelihood],
                            likelihood_name[face.sorrow_likelihood],
                            likelihood_name[face.joy_likelihood],
                            likelihood_name[face.under_exposed_likelihood],
                            likelihood_name[face.blurred_likelihood],
                            likelihood_name[face.headwear_likelihood]]

                for item, item2 in zip(emo, sentiment):
                    print (item, ": ", item2)

                if not (all( item == 'VERY_UNLIKELY' for item in sentiment) ):

                    if any( item == 'VERY_LIKELY' for item in sentiment):
                        state = sentiment.index('VERY_LIKELY')
                        # the order of enum type Likelihood is:
                        #'LIKELY', 'POSSIBLE', 'UNKNOWN', 'UNLIKELY', 'VERY_LIKELY', 'VERY_UNLIKELY'
                        # it makes sense to do argmin if VERY_LIKELY is not present, one would espect that VERY_LIKELY
                        # would be the first in the order, but that's not the case, so this special case must be added
                    else:
                        state = np.argmin(sentiment)

                    string = emo[state]

                cv2.putText(img_an,string, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)

    # save image
            cv2.imwrite(path_img_an, img_an)

        else:
            cv2.imwrite(path_img_an, img)



In [4]:
import cv2
import os
import re
import numpy

class VideoTransformer():

    def __init__(self, fps=30):
        self.fps = fps

    def video_to_frame(self, path_video, dir_frames):

        vidcap = cv2.VideoCapture(path_video)
        success,image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite(dir_frames + "/frame_" + str(count).zfill(10) + ".jpg", image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1


    def frame_to_video(self, dir_frames, path_video, fps):
        num_img = []
        images = []
        for img in os.listdir(dir_frames):
            if img.endswith(".png") or img.endswith(".jpg"):
                # using List comprehension + isdigit() +split() 
                # getting numbers from string  
                data = img.split("_")
                data = data[1].split(".")
                num = data[0]
                #print(num)
                num_img.append(num)
                images.append(img)

        sorted_numbers = numpy.argsort(num_img)
        images_sorted = []
        #print(images)
        for i in range(len(images)):
            images_sorted.append(images[sorted_numbers[i]])

        frame = cv2.imread(os.path.join(dir_frames, images_sorted[0]))
        height, width, layers = frame.shape
        fourrc = cv2.VideoWriter_fourcc(*'MP4V')
        video = cv2.VideoWriter(path_video, fourrc, fps, (width, height))

        for image in images_sorted:
            frame = cv2.imread(os.path.join(dir_frames, image))
            video.write(frame)

        cv2.destroyAllWindows()
        video.release()

In [6]:
import os

from datetime import datetime
start_time = datetime.now()

# Set parameters
PATH_CRED = "D://OD//live//VideoDetection-503abf24e493.json"
PATH_VID = "D://OD//live//sample.mp4"# location video 
DIR_FRAMES = "D://OD//live//frames//"# directory to store frames (will be created)
PATH_VID_AN = "D://OD//live//sample_720p_output.mp4"# name of annotated video
DIR_FRAMES_AN = "D://OD//live//labeled_frames//"# directory to store annotated frames (will be created)
VID_FPS = 30

print("running")

# Create objects
img_annotator = ImageAnnotator(PATH_CRED)
video_transf = VideoTransformer(VID_FPS)

# create folder for frames
try:
    os.mkdir(DIR_FRAMES)
except:
    print("Folder " + DIR_FRAMES + " already exists")
    for filename in os.listdir(DIR_FRAMES):
        file_path = os.path.join(DIR_FRAMES, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

try:
    os.mkdir(DIR_FRAMES_AN)
except:
    print("Folder " + DIR_FRAMES_AN + " already exists")
    for filename in os.listdir(DIR_FRAMES_AN):
        file_path = os.path.join(DIR_FRAMES_AN, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Turn video into frames
print("Turn video into frames")
video_transf.video_to_frame(PATH_VID, DIR_FRAMES)

# iterate through frames
print("Get informations from images")
for file in os.listdir(DIR_FRAMES):
    if file.endswith(".jpg") or file.endswith(".png"):
        # Get information from images
        img_objects = img_annotator.get_faces(DIR_FRAMES+file, True)

        # Annotate images
        img_annotator.annotate_image(DIR_FRAMES+file, DIR_FRAMES_AN+file, img_objects)

# transform frames to video
print("Transform frames to video")
video_transf.frame_to_video(DIR_FRAMES_AN, PATH_VID_AN,VID_FPS)

print("Done!")

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
